In [1]:
import import_ipynb
from maze_common import Maze, shortestPathSearch
import time
import random as rnd
import numpy as np
import copy

importing Jupyter notebook from maze_common.ipynb


In [17]:
maze = Maze(10, .3)
solvable = maze.isSolvable()
print(maze.board, "\nIs solvable? ", solvable)

[[ 0 -1 -1  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 -1]
 [ 0  0 -1 -1  0  0 -1  0 -1  0]
 [-1  0  0  0 -1  0  0 -1  0  0]
 [ 0  0 -1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0 -1 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0 -1  0  0 -1  0  0]
 [ 0 -1 -1  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0  0  0  0]] 
Is solvable?  True


In [18]:
def manhattanDistance(maze, cell):
    return abs(cell.row - (maze.dim - 1)) + abs(cell.col - (maze.dim - 1))

if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)
    

***** Found shortest path in 0.008975028991699219 seconds *****
[(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (3, 2), (3, 3), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (9, 9)]


In [19]:
def thinMaze(maze, fractionRemove):
    numRemove = round(fractionRemove * maze.obstacles.shape[0])
    thinMaze = copy.deepcopy(maze)
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX][obstacleY] = 0
    return thinMaze
    
thinnedMaze = thinMaze(maze, .6)
print("Original obstacles: ", maze.obstacles.shape[0], "\n", maze.obstacles)
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0], "\n",thinnedMaze.obstacles)
print("Thinned maze:\n", thinnedMaze.board)

Original obstacles:  21 
 [[0 1]
 [0 2]
 [0 6]
 [1 9]
 [2 2]
 [2 3]
 [2 6]
 [2 8]
 [3 0]
 [3 4]
 [3 7]
 [4 2]
 [5 2]
 [5 6]
 [5 7]
 [7 1]
 [7 4]
 [7 7]
 [8 1]
 [8 2]
 [8 9]]
Thinned obstacles:  8 
 [[0 6]
 [2 3]
 [2 6]
 [3 0]
 [3 7]
 [4 2]
 [5 2]
 [8 1]]
Thinned maze:
 [[ 0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0 -1  0  0  0]
 [-1  0  0  0  0  0  0 -1  0  0]
 [ 0  0 -1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]


In [22]:
visited = {}

def thinnedMazeShortestPathLength(maze, cell):
    shortestPathLength = 0
    if ((cell.row, cell.col)) not in visited:
        shortestPath = shortestPathSearch(thinnedMaze, startCoords = (cell.row, cell.col))
        shortestPathLength = len(shortestPath)
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cell.row, cell.col)]
    return shortestPathLength

if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)

***** Found shortest path in 2.817199945449829 seconds *****
[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (6, 6), (6, 7), (6, 8), (7, 8), (8, 8), (9, 8), (9, 9)]
